### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [9]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [10]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [11]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [12]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
32,42666497,18045,79085,200,0,0,1142,6,2025-05-30 19:25,2025-05-30 19:25,4,0,5686,1,1
33,42670728,79085,37239,51530,0,0,1143,0,2025-05-31 22:00,2025-05-31 22:00,6,0,2210,6,1
34,42410252,79085,79665,1349,21,70,1143,1,2025-06-01 21:59,2025-06-01 21:59,3,0,15959,7,1
35,42669966,4315,79085,51531,0,0,1143,2,2025-06-03 01:30,2025-06-03 01:30,3,0,3899,0,1
36,42669970,90579,79085,51530,0,0,1143,3,2025-06-03 18:00,2025-06-03 18:00,0,0,2218,1,1
37,42410240,79085,80293,1349,22,70,1143,4,2025-06-04 21:59,2025-06-04 21:59,1,2,11924,5,1
38,42670086,78043,79085,51531,0,0,1143,5,2025-06-05 11:00,2025-06-05 11:00,5,0,4396,5,1


In [13]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [14]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

7

### Estatísticas dos jogos

In [15]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [16]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,18045,20710,36530,2,14,2,0,0,god,12,10,9,42666497
1,79085,49268,33448,1,2,3,0,0,5-4-1a,6,8,8,42666497
0,79085,16241,45174,1,26,0,0,0,4-3-3aRU,7,8,8,42670728
1,37239,53740,24807,0,0,3,0,0,too ta,3,4,2,42670728
0,79085,32521,36957,4,12,1,1,0,4-2-4a,10,9,10,42410252
1,79665,37599,33163,1,4,3,0,0,hau,8,7,8,42410252
0,4315,29274,35075,1,19,1,1,0,crowns,10,10,11,42669966
1,79085,40674,34873,0,6,5,2,0,4-4-2c,9,9,10,42669966
0,90579,22898,32680,1,5,1,0,0,DEFENSIVA ALA E,9,10,11,42669970
1,79085,47178,37396,4,2,1,0,0,4-4-2c,8,9,10,42669970


In [17]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [18]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1782

### Estatísticas da Liga

In [19]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [20]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [21]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,10520,22,59,19,2,1,102,16,59586102019011,1349
1,23775,22,58,19,1,2,70,20,58550070019001,1349
2,124139,22,45,14,3,5,58,24,45534058014006,1349
3,23755,22,41,13,2,7,62,35,41527062013010,1349
4,122169,22,36,10,6,6,57,42,36515057010005,1349
5,78671,22,35,11,2,9,40,34,35506040011008,1349
6,79085,22,29,8,5,9,37,29,29508037008004,1349
7,79866,22,28,7,7,8,43,27,28516043007009,1349
8,89569,22,22,6,4,12,33,49,22484033006007,1349
9,80293,22,16,4,4,14,24,42,16482024004003,1349


In [22]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [23]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [24]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [25]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [26]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [27]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()